# RAG(Retrieval Augmented Generation)
- [RAG](https://python.langchain.com/v0.1/docs/modules/data_connection/)은 *Retrieval Augmented Generation*의 약자로, **검색 기반 생성 기법**을 의미한다. 이 기법은 LLM이 특정 문서에 기반하여 보다 정확하고 신뢰할 수 있는 답변을 생성할 수 있도록 돕는다.     
- 사용자의 질문에 대해 자체적으로 구축한 데이터베이스(DB)나 외부 데이터베이스에서 질문과 관련된 문서를 검색하고, 이를 질문과 함께 LLM에 전달한다.
- LLM은 같이 전달된 문서를 바탕으로 질문에 대한 답변을 생성한다. 
- 이를 통해 LLM이 학습하지 않은 내용도 다룰 수 있으며, 잘못된 정보를 생성하는 환각 현상(*hallucination*)을 줄일 수 있다.

## RAG와 파인튜닝(Fine Tuning) 비교

### 파인튜닝(Fine Tuning)

- **정의**: 사전 학습(pre-trained)된 LLM에 특정 도메인의 데이터를 추가로 학습시켜 해당 도메인에 특화된 맞춤형 모델로 만드는 방식이다.
- **장점**
  - 특정 도메인에 최적화되어 높은 정확도와 성능을 낼 수 있다.
- **단점**
  - 모델 재학습에 많은 시간과 자원이 필요하다.
  - 새로운 정보가 반영되지 않으며, 이를 위해서는 다시 학습해야 한다.

### RAG

- **정의**: 모델을 다시 학습시키지 않고, 외부 지식 기반에서 정보를 검색하여 실시간으로 답변에 활용하는 방식이다.
- **장점**
  - 최신 정보를 쉽게 반영할 수 있다.
  - 모델을 수정하지 않아도 되므로 효율적이다.
- **단점**
  - 검색된 문서의 품질에 따라 답변의 정확성이 달라질 수 있다.
  - 검색 시스템 구축이 필요하다.

## 정리

| 항목       | 파인튜닝 | RAG |
| -------- | ---- | --- |
| 도메인 최적화  | 가능   | 제한적 |
| 최신 정보 반영 | 불가능  | 가능  |
| 구현 난이도   | 높음   | 보통  |
| 유연성      | 낮음   | 높음  |

- LLM은 학습 당시의 데이터만을 기반으로 작동하므로 최신 정보나 기업 내부 자료와 같은 특정한 지식 기반에 접근할 수 없다.
- 파인튜닝은 시간과 비용이 많이 들고 유지보수가 어렵다.
-	반면, RAG는 기존 LLM을 변경하지 않고도 외부 문서를 통해 그 한계를 보완할 수 있다.
- RAG는 특히 빠르게 변화하는 정보를 다루는 분야(예: 기술 지원, 뉴스, 법률 등)에서 유용하게 활용된다. 반면, 정적인 정보에 대해 높은 정확도가 필요한 경우에는 파인튜닝이 효과적이다.


## RAG 작동 단계
- 크게 "**정보 저장(인덱싱)**", "**검색**, **생성**"의 단계로 나눌 수 있다.
  
### 1. 정보 저장(인덱싱)
RAG는 사전에 정보를 가공하여 **벡터 데이터베이스**(Vector 저장소)에 저장해 두고, 나중에 검색할 수 있도록 준비한다. 이 단계는 다음과 같은 과정으로 이루어진다.

1. **Load (불러오기)**
   - 답변시 참조할 사전 정보를 가진 데이터들을 불러온다.
2. **Split/Chunking (문서 분할)**
   - 긴 텍스트를 일정한 길이의 작은 덩어리(*chunk*)로 나눈다.
   - 이렇게 해야 검색과 생성의 정확도를 높일 수 있다.
3. **Embedding (임베딩)**
   - 각 텍스트 조각을 **임베딩 벡터**로 변환한다.
   - 임베딩 벡터는 그 문서의 의미를 벡터화 한 것으로 질문과 유사한 문서를 찾을 때 인덱스로 사용된다.
4. **Store (저장)**
   - 임베딩된 벡터를 **벡터 데이터베이스**(벡터 저장소)에 저장한다.
   - 벡터 데이터베이스는 유사한 질문이나 문장을 빠르게 찾을 수 있도록 특화된 데이터 저장소이다.
   
![rag](figures/rag1.png)

### 2. 검색, 생성

사용자가 질문을 하면 다음과 같은 절차로 답변이 생성된다.
1. **Retrieve (검색)**
   - 사용자의 질문을 임베딩한 후, 이 질문 벡터와 유사한 context 벡터를 벡터 데이터베이스에서 검색하여 찾는다.
2. **Query (질의 생성)**
   - 벡터 데이터베이스에서 검색된 문서 조각과 사용자의 질문을 함께 **프롬프트**(prompt)로 구성하여 LLM에 전달한다.
3. **Generation (응답 생성)**
   - LLM은 받은 프롬프트에 대한 응답을 생성한다.
   
- **RAG 흐름**
  
![Retrieve and Generation](figures/rag2.png)


# Document Loader
- LLM에게 질의할 때 같이 제공할 Data들을 저장하기 위해 먼저 읽어들인다.(Load)
- 데이터 Resouce는 다양하다.
    - 데이터를 로드(load)하는 방식은 저장된 위치와 형식에 따라 다양하다. 
      - 로컬 컴퓨터(Local Computer)에 저장된 문서
        - 예: CSV, Excel, JSON, TXT 파일 등
      - 데이터베이스(Database)에 저장된 데이터셋
      - 인터넷에 존재하는 데이터
        - 예: 웹에 공개된 API, 웹 페이지에 있는 데이터, 클라우드 스토리지에 저장된 파일 등

![rag_load](figures/rag_load.png)

- 다양한 문서 형식(format)에 맞춰 읽어오는 다양한 **document loader** 들을 Langchain에서 지원한다.
    - 다양한 Resource들로 부터 데이터를 읽기 위해서는 다양한 라이브러리를 이용해 서로 다른 방법으로 읽어야 한다.
    - Langchain은 데이터를 읽는 다양한 방식의 코드를 하나의 interface로 사용 할 수 있도록 지원한다.
        - https://python.langchain.com/docs/how_to/#document-loaders
    - 다양한 3rd party library(ppt, github 등등 다양한 3rd party lib도 있음. )들과 연동해 다양한 Resource로 부터 데이터를 Loading 할 수 있다.
        - https://python.langchain.com/docs/integrations/document_loaders/
- **모든 document loader는 기본적으로 동일한 interface(사용법)로 호출할 수있다.**
- **반환타입**
    - **list[Document]**
    - Load 한 문서는 Document객체에 정보들을 넣는다. 여러 문서를 읽을 수 있기 대문에 list에 묶어서 반환한다.
        - **Document 속성**
            - page_content: 문서의 내용
            - metadata(option): 문서에 대한 메타데이터(정보)를 dict 형태로 저장한다. 
            - id(option): 문서의 고유 id
     
- **주의**
    - Langchain을 이용해 RAG를 구현할 때 **꼭 Langchain의 DocumentLoader를 사용해야 하는 것은 아니다.**
    - DocumentLoader는 데이터를 읽어오는 것을 도와주는 라이브러리일 뿐이다. 다른 라이브러리를 이용해서 읽어 들여도 상관없다. 

## 주요 Document Loader

### Text file
- TextLoader 이용

In [4]:
from langchain_community.document_loaders import TextLoader

path = "data/olympic.txt"

# with open(path, 'rt') as f:
#     doc = f.read()

# 1. 객체 생성 -> 읽어올 자원의 정보(경로)를 제공.
loader = TextLoader(path, encoding="utf-8")

# 2. 읽어 오기(Loading)
docs = loader.load()  # lazy_load() -> 문서를 사용하는 시점에 읽어온다.

print(type(docs), len(docs))
print(type(docs[0]))

<class 'list'> 1
<class 'langchain_core.documents.base.Document'>


In [8]:
# Document 객체 속성
doc = docs[0]
print("문서의 정보-metadata:", doc.metadata)
print("문서식별자(ID):", doc.id)
print("문서내용:")
print(doc.page_content[:100])

문서의 정보-metadata: {'source': 'data/olympic.txt'}
문서식별자(ID): None
문서내용:
올림픽
올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하


In [ ]:
from langchain_core.documents import Document

with open(path, 'rt') as f:
    load_doc = f.read()

d = Document(page_content=load_doc, metadata={"category":"올림픽", "path":path})


Document(metadata={'category': '올림픽', 'path': 'data/olympic.txt'}, page_content='올림픽\n올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가 출신의 선수들이 참여하는 만큼 전 세계 스포츠 팬들이 가장 많이 시청하는 이벤트이다. 2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인류 역사상 가장 많은 수의 인구가 시청한 이벤트였다.\n또한 20세기에 올림픽 운동이 발전함에 따라, IOC는 변화하는 세계의 사회 환경에 적응해야 했다. 이러한 변화의 예로는 얼음과 눈을 이용한 경기 종목을 다루는 동계 올림픽, 장애인이 참여하는 

### PDF
- PyPDF, Pymupdf 등 다양한 PDF 문서를 읽어들이는 파이썬의  3rd party library들을 이용해 pdf 문서를 Load 한다.
    - https://python.langchain.com/docs/integrations/document_loaders/#pdfs
- 각 PDF Loader 특징
    -  PyMuPDFLoader
        -   텍스트 뿐 아니라 이미지, 주석등의 정보를 추출하는데 성능이 좋다.
        -   PyMuPDF 라이브러리 기반
    - PyPDFLoader
        - 텍스트를 빠르게 추출 할 수있다.
        - PyPDF2 라이브러리 기반. 경량 라이브러리로 빠르고 큰 파일도 효율적으로 처리한다.
    - PDFPlumberLoader
        - 표와 같은 복잡한 구조의 데이터 처리하는데 강력한 성능을 보여준다. 텍스트, 이미지, 표 등을 모두 추출할 수 있다. 
        - PDFPlumber 라이브러리 기반
- 설치 패키지
    - DocumentLoader와 연동하는 라이브러리들을 설치 해야 한다.
    - `pip install pypdf -qU`
    - `pip install pymupdf -qU`
    - `pip install pdfplumber -qU`

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# 1. 객체 생성 -> raw 데이터 연결
path = "data/novel/금_따는_콩밭_김유정.pdf"

loader = PyPDFLoader(path)

docs = loader.load()  # List[Document]
len(docs)  # 페이지당 하나의 문서(Document)

23

In [15]:
print(docs[1].page_content)

2 
위키백과
위키백과에  이  글
과  관련된 
자료가  있습니다 .
금  따는  콩밭
🙝 🙟 
땅속  저  밑은  늘  음침하
다 .
고달픈  간드렛불 , 맥없이
푸르끼하다 .
밤과  달라서  낮엔  되우  흐릿하였다 .
겉으로  황토  장벽으로  앞뒤좌우가  콕  막힌  좁직한  구뎅이 .
흡사히  무덤  속같이  귀중중하다 . 싸늘한  침묵 , 쿠더브레한
흙내와  징그러운  냉기만이  그  속에  자욱하다 .
곡괭이는  뻔질  흙을  이르집는다 . 암팡스러이  내려쪼며 ,
퍽  퍽  퍼억 .
이렇게  메떨어진  소리뿐 . 그러나  간간  우수수  하고  벽이  헐
린다 .
영식이는  일손을  놓고  소맷자락을  끌어당기어  얼굴의  땀을
훑는다 . 이놈의  줄이  언제나  잡힐는지  기가  찼다 . 흙  한줌을
집어  코밑에  바짝  들여대고  손가락으로  샅샅이  뒤져본다 . 완
연히  버력은  좀  변한  듯싶다 . 그러나  불통버력이  아주  다  풀
린  것도  아니었다 . 밀똥버력이라야  금이  온다는데  왜  이리
안  나오는지 .
곡괭이를  다시  집어든다 . 땅에  무릎을  꿇고  궁뎅이를  번쩍
든  채  식식거린다 . 곡괭이는  무작정  내려찍는다 . 바닥에서


In [17]:
docs[1].metadata

{'producer': 'Wikisource',
 'creator': 'Wikisource',
 'creationdate': '2024-11-24T07:05:35+00:00',
 'author': 'Unknown',
 'moddate': '2024-11-24T07:05:37+00:00',
 'title': '금 따는 콩밭',
 'source': 'data/novel/금_따는_콩밭_김유정.pdf',
 'total_pages': 23,
 'page': 1,
 'page_label': '2'}

In [18]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(path)

docs = loader.load()
len(docs)

23

In [20]:
print(docs[1].page_content)

2
위키백과
위키백과에 이 글
과 관련된
자료가 있습니다.
금 따는 콩밭
🙝🙟
땅속 저 밑은 늘 음침하
다.
고달픈 간드렛불, 맥없이
푸르끼하다.
밤과 달라서 낮엔 되우 흐릿하였다.
겉으로 황토 장벽으로 앞뒤좌우가 콕 막힌 좁직한 구뎅이.
흡사히 무덤 속같이 귀중중하다. 싸늘한 침묵, 쿠더브레한
흙내와 징그러운 냉기만이 그 속에 자욱하다.
곡괭이는 뻔질 흙을 이르집는다. 암팡스러이 내려쪼며,
퍽 퍽 퍼억.
이렇게 메떨어진 소리뿐. 그러나 간간 우수수 하고 벽이 헐
린다.
영식이는 일손을 놓고 소맷자락을 끌어당기어 얼굴의 땀을
훑는다. 이놈의 줄이 언제나 잡힐는지 기가 찼다. 흙 한줌을
집어 코밑에 바짝 들여대고 손가락으로 샅샅이 뒤져본다. 완
연히 버력은 좀 변한 듯싶다. 그러나 불통버력이 아주 다 풀
린 것도 아니었다. 밀똥버력이라야 금이 온다는데 왜 이리
안 나오는지.
곡괭이를 다시 집어든다. 땅에 무릎을 꿇고 궁뎅이를 번쩍
든 채 식식거린다. 곡괭이는 무작정 내려찍는다. 바닥에서


In [21]:
docs[0].metadata

{'producer': 'Wikisource',
 'creator': 'Wikisource',
 'creationdate': '2024-11-24T07:05:35+00:00',
 'source': 'data/novel/금_따는_콩밭_김유정.pdf',
 'file_path': 'data/novel/금_따는_콩밭_김유정.pdf',
 'total_pages': 23,
 'format': 'PDF 1.4',
 'title': '금 따는 콩밭',
 'author': 'Unknown',
 'subject': '',
 'keywords': '',
 'moddate': '2024-11-24T07:05:37+00:00',
 'trapped': '',
 'modDate': "D:20241124070537+00'00'",
 'creationDate': "D:20241124070535+00'00'",
 'page': 0}

### Web

- WebBaseLoader 이용
  - 입력받은 URL의 웹 문서를 읽어 문서로 로드한다. 웹 크롤링작업 없이 웹상의 문서를 가져올 수있다.
  - 내부적으로 BeautifulSoup을 이용해 웹문서를 parsing한다.
- https://python.langchain.com/docs/how_to/document_loader_web/

In [ ]:
%pip install bs4

In [27]:
from langchain_community.document_loaders import WebBaseLoader

url = [
    "https://m.sports.naver.com/wfootball/article/421/0008308548",
    "https://m.sports.naver.com/wfootball/article/450/0000131435"
]

my_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36"

loader = WebBaseLoader(
    web_path=url, # 개별 페이지 -> str, 여러페이지 -> list[str]
    header_template={
        "user-agent":my_user_agent
    }
)

docs = loader.load()
len(docs)

2

In [24]:
docs[0].metadata

{'source': 'https://m.sports.naver.com/wfootball/article/421/0008308548',
 'title': '쿠팡플레이, 스포츠패스 금액 월 1만 원 확정…15일부터 시행',
 'language': 'ko'}

In [26]:
print(docs[0].page_content)

쿠팡플레이, 스포츠패스 금액 월 1만 원 확정…15일부터 시행NAVER스포츠메뉴홈야구해외야구축구해외축구농구배구N골프일반e스포츠아웃도어NEW뉴스영상일정순위포토홈 바로가기NAVER스포츠마이팀팀 추가응원하는 팀을 구독해보세요!스포츠야구해외야구축구해외축구농구배구N골프일반e스포츠아웃도어콘텐츠오늘의 경기승부예측연재이슈톡대학스포츠랭킹기타고객센터공식 블로그메뉴 닫기본문 바로가기쿠팡플레이, 스포츠패스 금액 월 1만 원 확정…15일부터 시행입력2025.06.12. 오후 2:00수정2025.06.12. 오후 2:01기사원문김정현 기자양새롬 기자공감좋아요0슬퍼요0화나요0팬이에요0후속기사 원해요0텍스트 음성 변환 서비스본문 듣기를 종료하였습니다.글자 크기 변경공유하기쿠팡 와우 회원, 월 총 요금  '1만 7890원'(쿠팡플레이 갈무리)/뉴스1(서울=뉴스1) 김정현 양새롬 기자 = 쿠팡플레이가 부가서비스인 '스포츠 패스'의 금액을 월 1만 원으로 확정했다.12일 업계에 따르면 쿠팡플레이는 오는 15일 해외 스포츠 등의 콘텐츠를 유료 부가 서비스로 제공하는 스포츠 패스의 요금을 월 1만 원으로 결정했다. 쿠팡 와우 멤버십 구독료인 월 7890원에 스포츠패스 금액을 더하면 월 이용금액은 1만 7890원이 된다.이번 패스를 통해 볼수 있는 스포츠 리그는 △FIFA대회(FIFA클럽월드컵)△유럽축구리그(프리미어리그 2025~2026 시즌, 라리가, 분데스리가, 분데스리가2, 리그1, EFL 챔피언십 EFL리그원, 에레디비시) △유럽축구 토너먼트(FA컵, 카라바오컵, 커뮤니티쉴드, 코파 델레이, 수페르코파데 에스파냐, DFB-포칼, DFL-슈퍼컵, 쿠프드프랑스, 트로페데 샹피옹, 버투트로피) △아시아축구(AFC아시안컵, AFC챔피언스리그 엘리트, AFC챔피언스리그2, 기타AFC주관 국제 대회) △세계축구(월드컵남미 예선, 클럽 친선경기, 해외 국가 친선경기) 등이다.앞서 쿠팡플레이는 선택형 부가서비스 '패스(PASS)'를 6월 중 도입한다고 밝힌 바 있다.김정현 기자구독구독자 0응원수 0"리박스

- 페이지의 일부분만 가져오기.
- BeautifulSoup의 SoupStrainer 를 이용.
    - BeautifulSoup("html문서", parse_only=Strainer객체)
        - Strainer객체에 지정된 영역에서만 내용 찾는다.
    - Strainer("태그명") -> 지정한 태그 내에서만 찾는다.
    - Strainer(name="태그명", attrs={속성명:속성값}) -> 지정한 태그 중 속성명=속성값인 것 내에서만 찾는다.


In [ ]:
import bs4

loader = WebBaseLoader(
    web_path=url,
    # WebBaseLoader가 bs4를 사용. bs4에 전달할 파라미터를 설정하는 변수
    bs_kwargs={
        "parse_only":bs4.SoupStrainer(attrs={"class":"_article_content"})
    }
)

docs = loader.load()
len(docs)

2

In [34]:
print(docs[0].page_content)

쿠팡 와우 회원, 월 총 요금  '1만 7890원'(쿠팡플레이 갈무리)/뉴스1(서울=뉴스1) 김정현 양새롬 기자 = 쿠팡플레이가 부가서비스인 '스포츠 패스'의 금액을 월 1만 원으로 확정했다.12일 업계에 따르면 쿠팡플레이는 오는 15일 해외 스포츠 등의 콘텐츠를 유료 부가 서비스로 제공하는 스포츠 패스의 요금을 월 1만 원으로 결정했다. 쿠팡 와우 멤버십 구독료인 월 7890원에 스포츠패스 금액을 더하면 월 이용금액은 1만 7890원이 된다.이번 패스를 통해 볼수 있는 스포츠 리그는 △FIFA대회(FIFA클럽월드컵)△유럽축구리그(프리미어리그 2025~2026 시즌, 라리가, 분데스리가, 분데스리가2, 리그1, EFL 챔피언십 EFL리그원, 에레디비시) △유럽축구 토너먼트(FA컵, 카라바오컵, 커뮤니티쉴드, 코파 델레이, 수페르코파데 에스파냐, DFB-포칼, DFL-슈퍼컵, 쿠프드프랑스, 트로페데 샹피옹, 버투트로피) △아시아축구(AFC아시안컵, AFC챔피언스리그 엘리트, AFC챔피언스리그2, 기타AFC주관 국제 대회) △세계축구(월드컵남미 예선, 클럽 친선경기, 해외 국가 친선경기) 등이다.앞서 쿠팡플레이는 선택형 부가서비스 '패스(PASS)'를 6월 중 도입한다고 밝힌 바 있다.


### ArxivLoader
- https://github.com/lukasschwab/arxiv.py
- [arXiv-아카이브](https://arxiv.org/) 는 미국 코렐대학에서 운영하는 **무료 논문 저장소**로, 물리학, 수학, 컴퓨터 과학, 생물학, 금융, 경제 등 **과학, 금융 분야의 논문**들을 공유한다.
- `ArxivLoader` 를 사용해 원하는 주제의 논문들을 arXiv에서 가져와 load할 수 있다.
- **arXiv API**를 사용해 논문을 가져올 수 있다.
  - https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.arxiv.ArxivLoader.html
- 설치
  - `pip install langchain-community -qU`
  - `pip install arxiv -qU`



In [ ]:
import arxiv 

# 검색 기준 설정.
search = arxiv.Search(
    query="RAG", # 검색어
    max_results=2, # 검색 결과 최대 개수.
    sort_by=arxiv.SortCriterion.Relevance
)
# 정렬기준 - Relevance: 검색어 관련성이 높은 순서
#          - LastUpdatedDate: 논문이 마지막으로 수정된 날짜 기준.
#          - SubmittedDate: 처음 제출된 날짜 기준.

# 검색
client = arxiv.Client()
result = client.results(search)
print(type(result))

<class 'itertools.islice'>


In [37]:
doc1 = next(result)  # 첫번째 문서 for page in result:


In [42]:
print("논문제목:", doc1.title)
print("저자:", doc1.authors)
# print("요약: ", doc1.summary)
print("논문 PDF URL:", doc1.pdf_url)

논문제목: Modular RAG: Transforming RAG Systems into LEGO-like Reconfigurable Frameworks
저자: [arxiv.Result.Author('Yunfan Gao'), arxiv.Result.Author('Yun Xiong'), arxiv.Result.Author('Meng Wang'), arxiv.Result.Author('Haofen Wang')]
논문 PDF URL: http://arxiv.org/pdf/2407.21059v1


In [ ]:
# 다운로드
import os
os.makedirs("papers", exist_ok=True)

client = arxiv.Client()
result = client.results(search)

for idx, paper in enumerate(result, start=10):
    paper.download_pdf("papers", f"{idx}.pdf")
# doc1.download_pdf(다운받을 디렉토리, 파일명명)

In [ ]:
%pip install pymupdf

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.document_loaders import ArxivLoader

# load_max_docs=50 #
loader = ArxivLoader(
    query="Advanced RAG", 
    top_k_results=1, # 몇개 검색할지 지정.
)

docs = loader.load()
len(docs)

1

In [6]:
docs[0].metadata

{'Published': '2024-07-26',
 'Title': 'Modular RAG: Transforming RAG Systems into LEGO-like Reconfigurable Frameworks',
 'Authors': 'Yunfan Gao, Yun Xiong, Meng Wang, Haofen Wang',
 'Summary': 'Retrieval-augmented Generation (RAG) has markedly enhanced the capabilities\nof Large Language Models (LLMs) in tackling knowledge-intensive tasks. The\nincreasing demands of application scenarios have driven the evolution of RAG,\nleading to the integration of advanced retrievers, LLMs and other complementary\ntechnologies, which in turn has amplified the intricacy of RAG systems.\nHowever, the rapid advancements are outpacing the foundational RAG paradigm,\nwith many methods struggling to be unified under the process of\n"retrieve-then-generate". In this context, this paper examines the limitations\nof the existing RAG paradigm and introduces the modular RAG framework. By\ndecomposing complex RAG systems into independent modules and specialized\noperators, it facilitates a highly reconfigurabl

In [8]:
print(docs[0].page_content)

1
Modular RAG: Transforming RAG Systems into
LEGO-like Reconfigurable Frameworks
Yunfan Gao, Yun Xiong, Meng Wang, Haofen Wang
Abstract—Retrieval-augmented
Generation
(RAG)
has
markedly enhanced the capabilities of Large Language Models
(LLMs) in tackling knowledge-intensive tasks. The increasing
demands of application scenarios have driven the evolution
of RAG, leading to the integration of advanced retrievers,
LLMs and other complementary technologies, which in turn
has amplified the intricacy of RAG systems. However, the rapid
advancements are outpacing the foundational RAG paradigm,
with many methods struggling to be unified under the process
of “retrieve-then-generate”. In this context, this paper examines
the limitations of the existing RAG paradigm and introduces
the modular RAG framework. By decomposing complex RAG
systems into independent modules and specialized operators, it
facilitates a highly reconfigurable framework. Modular RAG
transcends the traditional linear architect

In [9]:
# 논문 요약만 조회
summary_docs = loader.get_summaries_as_docs()
print(summary_docs)

[Document(metadata={'Entry ID': 'http://arxiv.org/abs/2407.21059v1', 'Published': datetime.date(2024, 7, 26), 'Title': 'Modular RAG: Transforming RAG Systems into LEGO-like Reconfigurable Frameworks', 'Authors': 'Yunfan Gao, Yun Xiong, Meng Wang, Haofen Wang'}, page_content='Retrieval-augmented Generation (RAG) has markedly enhanced the capabilities\nof Large Language Models (LLMs) in tackling knowledge-intensive tasks. The\nincreasing demands of application scenarios have driven the evolution of RAG,\nleading to the integration of advanced retrievers, LLMs and other complementary\ntechnologies, which in turn has amplified the intricacy of RAG systems.\nHowever, the rapid advancements are outpacing the foundational RAG paradigm,\nwith many methods struggling to be unified under the process of\n"retrieve-then-generate". In this context, this paper examines the limitations\nof the existing RAG paradigm and introduces the modular RAG framework. By\ndecomposing complex RAG systems into ind

In [11]:
summary_docs[0].metadata

{'Entry ID': 'http://arxiv.org/abs/2407.21059v1',
 'Published': datetime.date(2024, 7, 26),
 'Title': 'Modular RAG: Transforming RAG Systems into LEGO-like Reconfigurable Frameworks',
 'Authors': 'Yunfan Gao, Yun Xiong, Meng Wang, Haofen Wang'}

In [12]:
print(summary_docs[0].page_content)

Retrieval-augmented Generation (RAG) has markedly enhanced the capabilities
of Large Language Models (LLMs) in tackling knowledge-intensive tasks. The
increasing demands of application scenarios have driven the evolution of RAG,
leading to the integration of advanced retrievers, LLMs and other complementary
technologies, which in turn has amplified the intricacy of RAG systems.
However, the rapid advancements are outpacing the foundational RAG paradigm,
with many methods struggling to be unified under the process of
"retrieve-then-generate". In this context, this paper examines the limitations
of the existing RAG paradigm and introduces the modular RAG framework. By
decomposing complex RAG systems into independent modules and specialized
operators, it facilitates a highly reconfigurable framework. Modular RAG
transcends the traditional linear architecture, embracing a more advanced
design that integrates routing, scheduling, and fusion mechanisms. Drawing on
extensive research, this pa

### Docling
- IBM Research에서 개발한 오픈소스 문서처리 도구로 다양한 종류의 문서를 구조화된 데이터로 변환해 생성형 AI에서 활용할 수있도록 지원한다.
- **주요기능**
  - PDF, DOCX, PPTX, XLSX, HTML, 이미지 등 여러 형식을 지원
  - PDF의 **페이지 레이아웃, 읽기 순서, 표 구조, 코드, 수식** 등을 분석하여 정확하게 읽어들인다.
  - OCR을 지원하여 스캔된 PDF나 이미지에서 텍스트를 추출할 수있다.
  - 읽어들인 내용을 markdown, html, json등 다양한 형식으로 출력해준다.
- 설치 : `pip install langchain-docling ipywidgets -qU` 
- 참조
  - docling 사이트: https://github.com/docling-project/docling
  - 랭체인-docling https://python.langchain.com/docs/integrations/document_loaders/docling/

In [13]:
%pip install langchain-docling ipywidgets -qU

Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'pylatexenc' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pylatexenc'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [1]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType

from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os
# huggingface-hub 로그인
login(os.getenv("HUGGINGFACE_API_KEY"))

In [9]:
path = "papers/1.pdf" #문서 경로. local file경로, url
path = "https://arxiv.org/pdf/2506.09669"

loader = DoclingLoader(file_path=path, export_type=ExportType.MARKDOWN)
docs = loader.load()
len(docs)

1

In [10]:
docs[0].metadata

{'source': 'https://arxiv.org/pdf/2506.09669'}

In [ ]:
# print(docs[0].page_content)
from IPython.display import Markdown

Markdown(docs[0].page_content)

### UnstructuredLoader
- 다양한 비정형 문서들을 읽어 오는 Unstrctured 를 사용해, 다양한 형식의 문서들을 load 해 RAG, 모델 파인튜닝에 적용할 수있게 한다.
  - 지원 파일 형식: "csv", "doc", "docx", "epub", "image", "md", "msg", "odt", "org", "pdf", "ppt", "pptx", "rtf", "rst", "tsv", "xlsx"
- **다양한 형식의 파일로 부터 text를 로딩**해야 할 경우 유용하다. 
- Local에 library를 설치해서 사용하거나,  Unstructured 가 제공하는 API service를 사용할 수 있다.
  - https://docs.unstructured.io
- 텍스트 파일, PDF, 이미지, HTML, XML, ms-office(word, ppt), epub 등 다양한 비정형 데이터 파일을 처리할 수 있다.
  - 설치, 지원 문서: https://docs.unstructured.io/open-source/installation/full-installation
  - Langchain 문서: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file

> - UnstructuredLoader PDF Load 시 Document 분할 기준
>     -  문서의 구조와 콘텐츠를 기반으로 텍스트를 분할해 Document에 넣는다.
>     -  분할 기준
>        - 헤더(Header): 문서의 제목이나 섹션 제목 등
>        - 본문 텍스트(NarrativeText): 일반적인 문단이나 설명문
>        - 표(Table): 데이터가 표 형식으로 구성된 부분
>        - 리스트(List): 순서가 있거나 없는 목록
>        - 이미지(Image): 사진이나 그래픽 요소

#### 설치할 프로그램
- poppler
  - pdf 파일을 text로 변환하기 위해 필요한 프로그램
  - windows: https://github.com/oschwartz10612/poppler-windows/releases/ 에서 최신 버전 다운로드 후 압축 풀어서 설치.
    - 환경변수 Path에 "설치경로\Library\bin" 을 추가. (설치 후 IDE를 다시 시작한다.)
  - macOS: `brew install poppler`
  - Linux: `sudo apt-get install poppler-utils`
- tesseract-ocr
  - OCR 라이브러리로 pdf 이미지를 text로 변환하기 위해 필요한 프로그램 
  - windows: https://github.com/UB-Mannheim/tesseract/wiki 에서 다운받아 설치. 
    - 환경변수 Path에 설치 경로("C:\Program Files\Tesseract-OCR") 추가 한다. (설치 후 IDE를 다시 시작한다.)
  - macOS: `brew install tesseract`
  - linux(unbuntu): `sudo apt install tesseract-ocr`
- 설치 할 패키지
  - **libmagic 설치**
      - windows: `pip install python-magic-bin -qU`
      - macOS: `brew install libmagic`
      - linux(ubuntu): `sudo apt-get install libmagic-dev`
  - `pip install "unstructured[pdf]" -qU`
      - 문서 형식별로 sub module을 설치한다. (pdf, docx ..)
      - 모든 sub module 설치: `pip install unstructured[all-docs]`
      - https://docs.unstructured.io/open-source/installation/full-installation
  - `pip install langchain-unstructured -qU`

In [1]:
%pip install langchain-unstructured

  Using cached langchain_unstructured-0.1.6-py3-none-any.whl.metadata (3.3 kB)
  Using cached onnxruntime-1.19.2-cp312-cp312-win_amd64.whl.metadata (4.7 kB)
Using cached langchain_unstructured-0.1.6-py3-none-any.whl (7.0 kB)
Using cached onnxruntime-1.19.2-cp312-cp312-win_amd64.whl (11.1 MB)

  Attempting uninstall: onnxruntime

    Found existing installation: onnxruntime 1.22.0

    Uninstalling onnxruntime-1.22.0:

      Successfully uninstalled onnxruntime-1.22.0

   ---------------------------------------- 0/2 [onnxruntime]
   ---------------------------------------- 0/2 [onnxruntime]
   ---------------------------------------- 0/2 [onnxruntime]
   ---------------------------------------- 0/2 [onnxruntime]
   ---------------------------------------- 0/2 [onnxruntime]
   ---------------------------------------- 0/2 [onnxruntime]
   ---------------------------------------- 2/2 [langchain-unstructured]

Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain_unstructured import UnstructuredLoader
# path = "data/olympic.txt"
# path = "papers/1.pdf"
path = ["data/olympic.txt", "papers/1.pdf"]
loader = UnstructuredLoader(path)
docs = loader.load()

In [15]:
len(docs)

465

In [17]:
docs[300].metadata

{'source': 'papers/1.pdf',
 'coordinates': {'points': ((70.866, 338.5113216),
   (70.866, 403.26892159999994),
   (290.7855161390001, 403.26892159999994),
   (290.7855161390001, 338.5113216)),
  'system': 'PixelSpace',
  'layout_width': 595.276,
  'layout_height': 841.89},
 'file_directory': 'papers',
 'filename': '1.pdf',
 'languages': ['eng'],
 'last_modified': '2025-06-12T15:51:33',
 'page_number': 10,
 'parent_id': 'c0ff9d37ae73259855ed24d509a77b06',
 'filetype': 'application/pdf',
 'category': 'NarrativeText',
 'element_id': 'e1d2951b77c3895facdfdac2c0f93dec'}

In [12]:
docs[10].page_content

'However, existing RAG methods have two limi- tations: (1) Limited RAG scenarios. Real-world RAG scenarios are complex: Given the query, the retrieved information may directly contain the an- swer, offer partial help, or be helpless. Some an- swers can be obtained from a single document, while others require multi-hop reasoning across multiple documents. Our preliminary study demon- strates existing RAG methods cannot adequately handle all such scenarios (Chan et al., 2024; Asai (2) Limited et al., 2024a; Liu et al., 2024b). task diversity. Due to the lack of a general RAG dataset, most current RAG methods (Wei et al., 2024; Zhang et al., 2024) are fine-tuned on task- specific datasets (e.g., NQ (Kwiatkowski et al., 2019), TrivialQA (Joshi et al., 2017)), which suffer from limited question diversity and data volume.'

### Directory 내의 문서파일들 로딩
- DirectoryLoader 이용

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(
    "data", # 읽어들일 문서들이 있는 디렉토리.
    recursive=True, # 하위디렉토리까지 검색할지 여부.
)
docs = loader.load()
len(docs)

In [21]:
idx = 13
docs[idx].metadata
docs[idx].page_content

'1\n\n술 권하는 사회\n\nExported from Wikisource on 2024년 11월 24일\n\n2\n\n🙝🙟 "아이그, 홀로 바느질을 하고 있던 아내는 얼굴을 살짝 찌푸 리고 가늘고 날카로운 소리로 부르짖었다. 바늘 끝이 왼손 엄지손가락 손톱 밑을 찔렀음이다. 그 손가락은 가늘게 떨고 하얀 손톱 밑으로 앵두빛 같은 피가 비친다.\n\n그것을 볼 사이도 없이 아내는 얼른 바늘을 빼고 다른 손 엄 지손가락으로 그 상처를 누르고 있다. 그러면서 하던 일가지 를 팔꿈치로 고이고이 밀어 내려놓았다. 이윽고 눌렀던 손을 떼어보았다. 그 언저리는 인제 다시 피가 아니 나려는 것처 럼 혈색이 없다 하더니, 그 희던 꺼풀 밑에 다시금 꽃물이 차 츰차츰 밀려온다.\n\n보일 듯 말 듯한 그 상처로부터 좁쌀 낟 같은 핏방울이 송송 솟는다. 또 아니 누를 수 없다. 이만하면 그 구멍이 아물었으 려니 하고 손을 떼면 또 얼마 아니되어 피가 비치어 나온다.\n\n인제 헝겊 오락지로 처매는 수밖에 없다. 그 상처를 누른채 그는 바느질고리에 눈을 주었다. 거기 쓸만한 오락지는 실패 밑에 있다. 그 실패를 밀어내고 그 오락지를 두 새끼손가락 사이에 집어올리려고 한동안 애를 썼다. 그 오락지는 마치 풀로 붙여둔 것같이 고리 밑에 착 달라붙어 세상 집혀지지 않는다. 그 두 손가락은 헛되이 그 오락지 위를 긁적거리고 있을 뿐이다.\n\n“왜 집혀지지를 않아!”\n\n그는 마침내 울 듯이 부르짖었다. 그리고 그것을 집어줄 사 람이 없나 하는 듯이 방안을 둘러보았다. 방안은 텅 비어 있\n\n3\n\n다. 어느 뉘 하나 없다. 호젓한 허영(虛影)만 그를 휘싸고 있 다. 바깥도 죽은 듯이 고요하다.\n\n시시로 퐁퐁 하고 떨어지는 수도의 물방울 소리가 쓸쓸하게 들릴 뿐. 문득 전등불이 광채(光彩)를 더하는 듯하였다. 벽 상(壁上)에 걸린 괘종(掛鍾)의 거울이 번들하며, 새로 한 점 을 가리키려는 시침(時針)이 위협하는 듯이 그의 눈을 쏜다. 그의 남편은 그때껏 돌아오지 않았었다.\n

In [ ]:
loader = DirectoryLoader(
    "data", # 읽어들일 문서들이 있는 디렉토리.
    glob=["*.txt"],   # 읽을 파일들의 확장자를 지정.
    recursive=False, # 하위디렉토리까지 검색할지 여부.
)
docs = loader.load()
len(docs)

3

In [23]:
len(docs)

15

In [28]:
docs[2].metadata

{'source': 'data\\restaurant_wine.txt'}

# Chunking (문서 분할)

![rag_split](figures/rag_split.png)

- Load 한 문서를 지정한 기준의 덩어리(chunk)로 나누는 작업을 진행한다.

## 나누는 이유
1. **임베딩 모델의 컨텍스트 길이 제한**
    - 대부분의 언어 모델은 한 번에 처리할 수 있는 토큰 수에 제한이 있다. 전체 문서를 통째로 입력하면 이 제한을 초과할 수 있어 처리가 불가능해진다.
2. **검색 정확도 향상**
    - 큰 문서 전체보다는 특정 주제나 내용을 다루는 작은 chunk가 사용자 질문과 더 정확하게 매칭된다. 예를 들어, 100페이지 매뉴얼에서 특정 기능에 대한 질문이 있을 때, 해당 기능을 설명하는 몇 개의 문단만 검색되는 것이 더 효과적이다.
    - 사용자 질문에 대해 문서의 모든 내용이 다 관련있는 것은 아니다. Chunking을 통해 가장 관련성 높은 부분만 선별적으로 활용할 수 있어 답변의 품질이 향상된다.
    - 전체 문서에는 질문과 무관한 내용들이 많이 포함되어 있어 모델이 혼란을 겪을 수 있다. 적절한 크기의 chunk는 이런 노이즈를 줄여준다.
3. **계산 효율성**
    - 벡터 유사도 계산, 임베딩 생성 등의 작업이 작은 chunk 단위로 수행될 때 더 빠르고 효율적이다. 메모리 사용량도 줄일 수 있다.

## 주요 Spliter
- https://api.python.langchain.com/en/latest/text_splitters_api_reference.html

### CharacterTextSplitter
가장  기본적인 Text spliter
- 한개의 구분자를 기준으로 분리한다. (default: "\n\n")
    - 분리된 조각이 chunk size 보다 작으면 다음 조각과 합칠 수 있다.
        - 합쳤을때 chuck_size 보다 크면 안 합친다. chuck_size 이내면 합친다.
    - 나누는 기준은 구분자이기 때문에 chunk_size 보다 글자수가 많을 수 있다.
- chunk size: 분리된 문서(chunk) 글자수 이내에서 분리되도록 한다.
    -  구분자를 기준으로 분리한다. 구분자를 기준으로 분리한 문서 조각이 chunk size 보다 크더라도 그대로 유지한다. 즉 chunk_size가 우선이 아니라 **seperator** 가 우선이다.
- 주요 파라미터
    - chunk_size: 각 조각의 최대 길이를 지정.
    - seperator: 구분 문자열을 지정. (default: '\n\n')
- CharacterTextSplitter는 단순 스플리터로 overlap기능을 지원하지는 않는다. 단 seperator가 빈문자열("") 일 경우에는 overlap 기능을 지원한다. overlap이란 각 이전 청크의 뒷부분의 문자열을 앞에 붙여 문맥을 유지하는 것을 말한다.
  
### RecursiveCharacterTextSplitter
- RecursiveCharacterTextSplitter는 **긴 텍스트를 지정된 최대 길이(chunk_size) 이하로 나누는 데 효과적인 텍스트 분할기**(splitter)이다.
- 여러 **구분자(separators)를 순차적으로 적용**하여, 가능한 한 자연스러운 문단/문장/단어 단위로 분할하고, 최종적으로는 크기 제한을 만족시킨다.
- 분할 기준 문자
    1. 두 개의 줄바꿈 문자 ("\n\n")
    2. 한 개의 줄바꿈 문자 ("\n")
    3. 공백 문자 (" ")
    4. 빈 문자열 ("")
- 작동 방식
    1. 먼저 가장 높은 우선순위의 구분자("\n\n")로 분할을 시도한다.
    2. 분할된 조각 중 **chunk_size를 초과하는 조각**에 대해 다음 우선순위 구분자("\n" → " " → "")로 재귀적으로 재분할한다.
    3. 이 과정을 통해 모든 조각(chunk)이 chunk_size를 초과하지 않도록 만든다.  
- 주요 파라미터
    - chunk_size: 각 조각의 최대 길이를 지정.
    - chunk_overlap: 연속된 청크들 간의 겹치는 문자 수를 설정. 새로운 청크 생성 시 이전 청크의 마지막 부분에서 지정된 수만큼의 문자를 가져와서 새 청크의 앞부분에 포함시켜, 청크 경계에서 문맥의 연속성을 유지한다.
      - 구분자에 의해 청크가 나눠지면 정상적인 분리이므로 overlap이 적용되지 않는다.
      - 정상적 구분자로 나눌 수 없어 chunk_size에 맞춰 잘라진 경우 문맥의 연결성을 위애 overlap을 적용한다.
    - separators(list): 구분자를 지정한다. 지정하면 기본 구분자가 지정한 것으로 변경된다.

#### 메소드
- `split_documents(Iterable[Document]) : List[Document]`
    - Document 목록을 받아 split 처리한다.
- `split_text(str) : List[str]`
    - string text를 받아서 split 처리한다. 

In [ ]:
text = """가각간갇갈갉갊감갑값갓갔강갖갗같갚갛개객갠갤갬갭갯갰

aadlskfjadklsfjakldfjadklsjadfskl갸갹갼걀걋걍걔걘걜거걱건걷걸걺검겁것겉겊겋게겐

띱띳띵라락란랄람랍랏랐

랑랒랖랗래랙랜랠램랩랫랬랭랴략랸럇량러럭런럴럼럽럿렀렁렇레렉렌렐렘렙렛렝나낙낚ASDFFGHJJKKLLLQWE

멨멩며 

멱면멸몃몄명몇몌모목몫몬몰몲몸몹못몽뫄뫈뫘뫙뫼묀묄묍묏묑묘묜묠묩묫무묵묶문묻물묽묾뭄뭅뭇뭉뭍뭏ABCDEFGHIJ"""

In [8]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

# chunk_size보다 chunk_overlap은 작아야 한다.
splitter = CharacterTextSplitter(
    chunk_size=60, chunk_overlap=10
    # , separator=""   # default:"\n\n"
)

docs = splitter.split_text(text)
len(docs), type(docs)

(6, list)

In [9]:
for doc in docs:
    print(len(doc), doc, sep=" || ")
    print("-------------------")

26 || 가각간갇갈갉갊감갑값갓갔강갖갗같갚갛개객갠갤갬갭갯갰
-------------------
56 || aadlskfjadklsfjakldfjadklsjadfskl갸갹갼걀걋걍걔걘걜거걱건걷걸걺검겁것겉겊겋게겐
-------------------
11 || 띱띳띵라락란랄람랍랏랐
-------------------
56 || 랑랒랖랗래랙랜랠램랩랫랬랭랴략랸럇량러럭런럴럼럽럿렀렁렇레렉렌렐렘렙렛렝나낙낚ASDFFGHJJKKLLLQWE
-------------------
3 || 멨멩며
-------------------
57 || 멱면멸몃몄명몇몌모목몫몬몰몲몸몹못몽뫄뫈뫘뫙뫼묀묄묍묏묑묘묜묠묩묫무묵묶문묻물묽묾뭄뭅뭇뭉뭍뭏ABCDEFGHIJ
-------------------


In [12]:
from langchain_core.documents import Document
document = Document(page_content=text)
docs2 = splitter.split_documents([document])
type(docs2), len(docs2), type(docs2[0])

(list, 6, langchain_core.documents.base.Document)

In [13]:
for d in docs2:
    print(d)

page_content='가각간갇갈갉갊감갑값갓갔강갖갗같갚갛개객갠갤갬갭갯갰'
page_content='aadlskfjadklsfjakldfjadklsjadfskl갸갹갼걀걋걍걔걘걜거걱건걷걸걺검겁것겉겊겋게겐'
page_content='띱띳띵라락란랄람랍랏랐'
page_content='랑랒랖랗래랙랜랠램랩랫랬랭랴략랸럇량러럭런럴럼럽럿렀렁렇레렉렌렐렘렙렛렝나낙낚ASDFFGHJJKKLLLQWE'
page_content='멨멩며'
page_content='멱면멸몃몄명몇몌모목몫몬몰몲몸몹못몽뫄뫈뫘뫙뫼묀묄묍묏묑묘묜묠묩묫무묵묶문묻물묽묾뭄뭅뭇뭉뭍뭏ABCDEFGHIJ'


In [14]:
splitter2 = RecursiveCharacterTextSplitter(
    chunk_size=50, chunk_overlap=10
    # , separators=["첫번째구분자", "두번째구분자", "세번째", ....]
    # default: ["\n\n", "\n", " ", ""]
)

result = splitter2.split_text(text)
print(type(result), len(result))

<class 'list'> 9


In [16]:
for r in result:
    print(len(r), r, sep="||")
    print("="* 70)

26||가각간갇갈갉갊감갑값갓갔강갖갗같갚갛개객갠갤갬갭갯갰
49||aadlskfjadklsfjakldfjadklsjadfskl갸갹갼걀걋걍걔걘걜거걱건걷걸걺검
17||걔걘걜거걱건걷걸걺검겁것겉겊겋게겐
11||띱띳띵라락란랄람랍랏랐
49||랑랒랖랗래랙랜랠램랩랫랬랭랴략랸럇량러럭런럴럼럽럿렀렁렇레렉렌렐렘렙렛렝나낙낚ASDFFGHJJK
17||ASDFFGHJJKKLLLQWE
3||멨멩며
49||멱면멸몃몄명몇몌모목몫몬몰몲몸몹못몽뫄뫈뫘뫙뫼묀묄묍묏묑묘묜묠묩묫무묵묶문묻물묽묾뭄뭅뭇뭉뭍뭏AB
18||묽묾뭄뭅뭇뭉뭍뭏ABCDEFGHIJ


In [19]:
result2 = splitter2.split_documents([document])
len(result2)

9

In [20]:
result2

[Document(metadata={}, page_content='가각간갇갈갉갊감갑값갓갔강갖갗같갚갛개객갠갤갬갭갯갰'),
 Document(metadata={}, page_content='aadlskfjadklsfjakldfjadklsjadfskl갸갹갼걀걋걍걔걘걜거걱건걷걸걺검'),
 Document(metadata={}, page_content='걔걘걜거걱건걷걸걺검겁것겉겊겋게겐'),
 Document(metadata={}, page_content='띱띳띵라락란랄람랍랏랐'),
 Document(metadata={}, page_content='랑랒랖랗래랙랜랠램랩랫랬랭랴략랸럇량러럭런럴럼럽럿렀렁렇레렉렌렐렘렙렛렝나낙낚ASDFFGHJJK'),
 Document(metadata={}, page_content='ASDFFGHJJKKLLLQWE'),
 Document(metadata={}, page_content='멨멩며'),
 Document(metadata={}, page_content='멱면멸몃몄명몇몌모목몫몬몰몲몸몹못몽뫄뫈뫘뫙뫼묀묄묍묏묑묘묜묠묩묫무묵묶문묻물묽묾뭄뭅뭇뭉뭍뭏AB'),
 Document(metadata={}, page_content='묽묾뭄뭅뭇뭉뭍뭏ABCDEFGHIJ')]

In [ ]:
##########################################
# olympic.txt 를 읽어서 split 처리
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. 문서 Load
path = "data/olympic.txt"
loader = TextLoader(path, encoding="utf-8")
docs = loader.load() # docs: list[Document]

# 2. load 한 문서를 Split
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = splitter.split_documents(docs)

print(len(split_docs)) # list[Document]

61


In [25]:
len_list = [len(d.page_content) for d in split_docs] # split된 문서들의 글자수
len_list

[3,
 498,
 403,
 414,
 239,
 262,
 5,
 496,
 222,
 270,
 338,
 310,
 434,
 456,
 498,
 5,
 496,
 135,
 446,
 413,
 4,
 498,
 120,
 356,
 233,
 400,
 392,
 310,
 221,
 496,
 226,
 428,
 362,
 495,
 379,
 311,
 355,
 268,
 405,
 2,
 495,
 495,
 242,
 362,
 493,
 374,
 236,
 329,
 297,
 459,
 498,
 154,
 401,
 444,
 466,
 352,
 499,
 111,
 10,
 498,
 217]

In [29]:
idx = 2
print(split_docs[idx].page_content)

1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가 출신의 선수들이 참여하는 만큼 전 세계 스포츠 팬들이 가장 많이 시청하는 이벤트이다. 2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인류 역사상 가장 많은 수의 인구가 시청한 이벤트였다.


In [30]:
from langchain_community.document_loaders import PyPDFLoader

# 문서 로드
path = "data/novel/메밀꽃_필_무렵_이효석.pdf"
loader = PyPDFLoader(path)
docs = loader.load()

# split
split_doc = splitter.split_documents(docs)
len(split_doc)

39

In [34]:
idx = 10
print(len(split_doc[idx].page_content), split_doc[idx].page_content)

494 6 
자라나기는  틀렸고  닳아 버린  철  사이로는  피가  빼짓이  흘렀
다 . 냄새만  맡고도  주인을  분간하였 다 . 호소하는  목소리로
야 단스럽게  울며  반겨한다 .
어 린아 이를  달래듯이  목덜미를  어 루만져주니  나귀는  코를
벌름거리고  입을  투르르거렸다 . 콧물이  튀었 다 . 허  생원은
짐승  때문에  속도  무던히는  썩였 다 . 아 이들의  장난이  심한
눈치여 서  땀  밴  몸뚱어 리가  부들부들  떨리고  좀체  흥분이  식
지  않는  모양 이었 다 . 굴레가  벗어 지고  안장도  떨어 졌다 . “ 요
몹쓸  자식들 ” 하고  허  생원은  호령을  하였 으나  패들은  벌써
줄행랑을  논  뒤요  몇  남지  않은  아 이들이  호령에  놀래  비슬
비슬  멀어 졌다 .
“ 우리들  장난이  아 니우 , 암 놈을  보고  저  혼자  발광이지 .”
코흘리개  한  녀석이  멀리서  소리를  쳤다 .
“ 고  녀석  말투가 ….”


In [35]:
# 문서 load 와 split을 동시에 처리.
split_docs = loader.load_and_split(splitter)
len(split_docs)

39

## Token 수 기준으로 나누기

- LLM 언어 모델들은 입력 토큰 수 제한이 있어서 요청시 제한 토큰수 이상의 프롬프트는 전송할 수 없다.
- 따라서 텍스트를 chunk로 분할할 때는 글자수 보다 **토큰 수를 기준으로 크기를 지정하는 것**이 좋다.  
- 토큰기반 분할은 텍스트의 의미를 유지하면서 분할하는 방식이므로 문자 기반 분할과 같이 단어가 중간잘리는 것들을 방지할 수 있다. 
- 토큰 수 계산할 때는 사용하는 언어 모델에 사용된 것과 동일한 tokenizer를 사용하는 것이 좋다.
  - 예를 들어 OpenAI의 GPT 모델을 사용할 경우 tiktoken 라이브러리를 활용하여 토큰 수를 정확하게 계산할 수 있다.

### [tiktoken](https://github.com/openai/tiktoken) tokenizer 기반 분할
- OpenAI에서 GPT 모델을 학습할 때 사용한 `BPE` 방식의 tokenizer. **OpenAI 언어모델을 사용할 경우 이것을 사용하는 것이 좀 더 정확하게  토큰dmf 계산할 수 있다.**
- Splitter.from_tiktoken_encoder() 메소드를 이용해 생성
  - `RecursiveCharacterTextSplitter.from_tiktoken_encoder()`
  - `CharacterTextSplitter.from_tiktoken_encoder()`
- 파라미터
  - encode_name: 인코딩 방식(토큰화 규칙)을 지정. OpenAI는 GPT 모델들 마다 다른 방식을 사용했다. 그래서 사용하려는 모델에 맞는 인코딩 방식을 지정해야 한다.
    - `cl100k_base`: GPT-4 및 GPT-3.5-Turbo 모델에서 사용된 방식.
    - `r50k_base:` GPT-3 모델에서 사용된 방식 
  - chunk_size, chunk_overlap, separators 파라미터 (위와 동일)
- tiktoken 설치
  - `pip install tiktoken`

### HuggingFace Tokenizer
- HuggingFace 모델을 사용할 경우 그 모델이 사용한 tokenizer를 이용해 토큰 기반으로 분할 한다.
  - 다른 tokenizer를 이용해 분할 할 경우 토큰 수 계산이 다르게 될 수있다.
- `from_huggingface_tokenizer()` 메소드를 이용.
  - 파라미터
    - tokenizer: HuggingFace tokenizer 객체
    - chunk_size, chunk_overlap, separators 파라미터 (위와 동일)
- `transformers` 라이브러리를 설치해야 한다.
  - `pip install transformers` 

In [36]:
%pip install tiktoken transformers

Note: you may need to restart the kernel to use updated packages.


In [40]:
loader = TextLoader("data/olympic.txt", encoding="utf-8")

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini", # 지정한 모델을 학습할때 사용한 토크나이저를 사용.
    chunk_size=500,  # 토큰수 기준.
    chunk_overlap=0
)

docs = loader.load_and_split(splitter)
len(docs)

36

In [41]:
idx = 1
print(docs[idx].page_content)

또한 20세기에 올림픽 운동이 발전함에 따라, IOC는 변화하는 세계의 사회 환경에 적응해야 했다. 이러한 변화의 예로는 얼음과 눈을 이용한 경기 종목을 다루는 동계 올림픽, 장애인이 참여하는 패럴림픽, 스페셜 올림픽, 데플림픽, 10대 선수들이 참여하는 유스 올림픽 등을 들 수 있다. 그 뿐만 아니라 IOC는 20세기의 변화하는 경제, 정치, 기술 환경에도 적응해야 했다. 그리하여 올림픽은 피에르 드 쿠베르탱이 기대했던 순수한 아마추어 정신에서 벗어나서, 프로 선수도 참가할 수 있게 되었다. 올림픽은 점차 대중 매체의 중요성이 커짐에 따라 올림픽의 상업화와 기업 후원을 놓고도 논란이 생겨났다. 또한 올림픽을 치르며 발생한 보이콧, 도핑, 심판 매수, 테러와 같은 수많은 일들은 올림픽이 더욱 굳건히 성장할 수 있는 원동력이 되었다.
올림픽은 국제경기연맹(IF), 국가 올림픽 위원회(NOC), 각 올림픽의 위원회(예-벤쿠버동계올림픽조직위원회)로 구성된다. 의사 결정 기구인 IOC는 올림픽 개최 도시를 선정하며, 각 올림픽 대회마다 열리는 올림픽 종목도 IOC에서 결정한다. 올림픽 경기 개최 도시는 경기 축하 의식이 올림픽 헌장에 부합하도록 조직하고 기금을 마련해야 한다. 올림픽 축하 행사로는 여러 의식과 상징을 들 수 있는데 올림픽기나 성화가 그 예이다.


In [42]:
# Huggingface Tokenizer
from transformers import AutoTokenizer
model_id = "beomi/kcbert-base"  # 사용할 LLM 모델의 ID
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [44]:
splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=300, 
    chunk_overlap=0
)
docs = loader.load_and_split(splitter)
len(docs)

47

In [46]:
idx = 10
print(docs[idx].page_content)

올림피아 경기는 기원전 6세기~기원전 5세기에 절정에 이르렀으나, 그 후 로마가 패권을 잡은 뒤 그리스에 영향력을 행사하면서 서서히 쇠퇴하게 된다. 고대 올림픽이 공식적으로 끝난 해는 확실히 알 수 없으나, 대부분 테오도시우스 1세 황제가 모든 이단 숭배 및 예배를 금지했던 393년을 고대 올림픽의 마지막이라고 추정한다. 다른 설에 따르면 테오도시우스의 후계자인 테오도시우스 2세가 모든 그리스 신전을 파괴하라고 명령한 426년이라고도 한다. 이렇게 올림픽이 사라진 이후로 이보다 한참 뒤인 19세기에 이르러서야 비로소 다시 올림픽 경기가 열리게 된다.


In [ ]:
# from langchain.text_splitter import 